LTOD calculations

Take 3 days worth of gsx input swath data for a sensor
    for each swath locate all measurements that lie between 70 and 71 N (or S) latitude
    save the longitude and UTC for each measurement
    
Plot a histogram of the times - but it's more complicated than that - 
    for each time, which is in UTC, take the longitude and calculate what the ltod is for that
    UTC at that longitude.
    
    i.e. if the UTC is 02:48 and the longitude is -105, then the ltod is ((2*60)+48) + (-105*4)


In [ ]:
#from __future__ import print_function
%pylab inline
from netCDF4 import Dataset
import matplotlib.pyplot as plt
import numpy as np
import glob as glob
import time
import calendar
import numpy.ma as ma

In [ ]:
# Set the directory locations for your input GSX files
amsr2_dir=
f18_dir=

In [ ]:
def seconds_to_midnight(seconds):
    """ This function calculates the number of seconds to midnight of the input
        time.  This is used to convert the scan times in the file to local time
        for that day """
    st = time.strptime("01.01.1987", "%d.%m.%Y")
    seconds_to_epoch = calendar.timegm(st)
    # get file epoch date in seconds since 1970
    str_date = time.gmtime(seconds + seconds_to_epoch)
    # get file date from number of seconds since 1987
    dmid = "%02d.%02d.%d" % (str_date.tm_mday, str_date.tm_mon, str_date.tm_year)
    utc_midnight = time.strptime(dmid, "%d.%m.%Y")
    utc_seconds_to_midnight = calendar.timegm(utc_midnight)
    # get midnight of file date in seconds
    return (utc_seconds_to_midnight - seconds_to_epoch)

In [ ]:
def create_ltod(files, lat_min=70., lat_max=71.):
    """ This function takes a list of gsx swath files as an argument and
        calculates the local time of each measurement in a specified latitude band
        It returns a 1-D array of the local times (in hours wrt midnight), 
        suitable for creating a histogram """
    
    fid = Dataset(files[0], 'r', 'NETCDF4')
    lat = np.array(fid.variables['latitude_loc1'])
    lon = np.array(fid.variables['longitude_loc1'])
    scantime = np.array(fid.variables['scan_time_loc1'])
    meas = len(fid.dimensions['measurements_loc1'])
    scan_time = np.array([scantime,]*meas).transpose()
    lat_all = lat.flatten()
    lon_all = lon.flatten()
    offset_seconds = seconds_to_midnight(scantime[0]) # get the offset to midnight for the first scanline
    scantime_all = scan_time.flatten() - offset_seconds 
    fid.close()
    for fil in files[1:-1]:
        #print (fil)
        fid = Dataset(fil, 'r', 'NETCDF4')
        scantime = np.array(fid.variables['scan_time_loc1'])
        if (len(scantime)):
            meas = len(fid.dimensions['measurements_loc1'])
            scan_time = np.array([scantime,]*meas).transpose()
            offset_seconds = seconds_to_midnight(scantime[0])
            scantime_all = np.concatenate((scantime_all, (scan_time.flatten() - offset_seconds)))
        else:
            fid.close()
            continue         
        lat = np.array(fid.variables['latitude_loc1'])
        lon = np.array(fid.variables['longitude_loc1'])
        lat_all = np.concatenate((lat_all, lat.flatten()))
        lon_all = np.concatenate((lon_all, lon.flatten()))
        fid.close()
    latm = ma.masked_where((lat_all < lat_min), lat_all)
    lat_band = ma.masked_where( latm > lat_max, latm)
    lon_band = ma.masked_array(lon_all, lat_band.mask)
    scantime_band = ma.masked_array(scantime_all, lat_band.mask)
    final_times = scantime_band + (lon_band*4*60.0)
    ftimes = final_times/3600.
    ftimes[ftimes > 24.0] = ftimes[ftimes > 24.0] - 24.0
    ftimes[ftimes < 0.] = ftimes[ftimes < 0.] + 24.0
    return ftimes

In [ ]:
A2_2024 = sort(glob.glob('%s/*2024070*.nc' % amsr2_dir))
len(A2_2024)

In [ ]:
ftn_A2_2024 = create_ltod(A2_2024, lat_min = 70.0, lat_max = 71.0)
fts_A2_2024 = create_ltod(A2_2024, lat_max = -70.0, lat_min = -71.0)



In [ ]:
fig = plt.figure(figsize=(15,21))
plt.suptitle('LTOD calculations based on observations between 70 and 71 N/S latitudes')
a = fig.add_subplot(1,2,1)
plt.hist(fts_A2_2024,bins=(np.arange(25)))
plt.plot([24, 24], [0, 40000], color='r', linestyle='--')
plt.plot([12, 12], [0, 40000], color='r', linestyle='--')
a.set_title('AMSR2 North - 1200/2400\n2024 doy 183-191\n127 files')
a = fig.add_subplot(1,2,2)
plt.hist(fts_A2_2024,bins=(np.arange(25)))
plt.plot([24, 24], [0, 40000], color='r', linestyle='--')
plt.plot([12, 12], [0, 40000], color='r', linestyle='--')
a.set_title('AMSR2 South - 1200/2400\n2021 doy 183-191\n127 files')


In [ ]:
F18_2022 = sort(glob.glob('%s/*2022070*.nc' % f18_dir))
len(F18_2022)

In [ ]:
F18_2021 = sort(glob.glob('%s/*202107*.nc' % f18_dir))
F18_2023 = sort(glob.glob('%s/*2023*.nc' % f18_dir))
print(len(F18_2021), len(F18_2023))

In [ ]:
ftn_F18_2022 = create_ltod(F18_2022, lat_min = 70.0, lat_max = 71.0)
fts_F18_2022 = create_ltod(F18_2022, lat_max = -70.0, lat_min = -71.0)
ftn_F18_2021 = create_ltod(F18_2021, lat_min = 70.0, lat_max = 71.0)
fts_F18_2021 = create_ltod(F18_2021, lat_max = -70.0, lat_min = -71.0)
ftn_F18_2023 = create_ltod(F18_2023, lat_min = 70.0, lat_max = 71.0)
fts_F18_2023 = create_ltod(F18_2023, lat_max = -70.0, lat_min = -71.0)


In [ ]:
fig = plt.figure(figsize=(15,21))
plt.suptitle('LTOD calculations based on observations between 70 and 71 N/S latitudes')
a = fig.add_subplot(3,2,1)
plt.hist(ftn_F18_2021,bins=(np.arange(25)))
plt.plot([24, 24], [0, 40000], color='r', linestyle='--')
plt.plot([12, 12], [0, 40000], color='r', linestyle='--')
a.set_title('F18 North - 1200/2400\n2021 doy 183-191\n127 files')
a = fig.add_subplot(3,2,2)
plt.hist(fts_F18_2021,bins=(np.arange(25)))
plt.plot([24, 24], [0, 40000], color='r', linestyle='--')
plt.plot([12, 12], [0, 40000], color='r', linestyle='--')
a.set_title('F18 South - 1200/2400\n2021 doy 183-191\n127 files')
a = fig.add_subplot(3,2,3)
plt.hist(ftn_F18_2022,bins=(np.arange(25)))
plt.plot([24, 24], [0, 40000], color='r', linestyle='--')
plt.plot([12, 12], [0, 40000], color='r', linestyle='--')
a.set_title('F18 North - 1200/2400\n2022 doy 183-191\n127 files')
a = fig.add_subplot(3,2,4)
plt.hist(fts_F18_2022,bins=(np.arange(25)))
plt.plot([24, 24], [0, 40000], color='r', linestyle='--')
plt.plot([12, 12], [0, 40000], color='r', linestyle='--')
a.set_title('F18 South - 1200/2400\n2022 doy 183-191\n127 files')
a = fig.add_subplot(3,2,5)
plt.hist(ftn_F18_2023,bins=(np.arange(25)))
plt.plot([24, 24], [0, 40000], color='r', linestyle='--')
plt.plot([12, 12], [0, 40000], color='r', linestyle='--')
a.set_title('F18 North - 1200/2400\n2023 doy 183-191\n127 files')
a = fig.add_subplot(3,2,6)
plt.hist(fts_F18_2023,bins=(np.arange(25)))
plt.plot([24, 24], [0, 40000], color='r', linestyle='--')
plt.plot([12, 12], [0, 40000], color='r', linestyle='--')
a.set_title('F18 South - 1200/2400\n2023 doy 183-191\n127 files')

plt.show()